In [ ]:
import pandas as pd

Mengimpor modul Python pandas untuk manajemen dan analisis data yang akan digunakan. Kemudian memanggil dataset movie ke dalam DataFrame.

In [ ]:
df = pd.read_csv('/content/movie_sample_dataset (1).csv')
df.head()

,color,director_name,duration,gross,genres,movie_title,title_year,language,country,budget,imdb_score,actors,movie_facebook_likes
0,Color,Martin Scorsese,240,116866727.0,Biography|Comedy|Crime|Drama,The Wolf of Wall Street,2013,English,USA,100000000.0,8.2,"Leonardo DiCaprio,Matthew McConaughey,Jon Favreau",138000
1,Color,Shane Black,195,408992272.0,Action|Adventure|Sci-Fi,Iron Man 3,2013,English,USA,200000000.0,7.2,"Robert Downey Jr.,Jon Favreau,Don Cheadle",95000
2,color,Quentin Tarantino,187,54116191.0,Crime|Drama|Mystery|Thriller|Western,The Hateful Eight,2015,English,USA,44000000.0,7.9,"Craig Stark,Jennifer Jason Leigh,Zoë Bell",114000
3,Color,Kenneth Lonergan,186,46495.0,Drama,Margaret,2011,English,usa,14000000.0,6.5,"Matt Damon,Kieran Culkin,John Gallagher Jr.",0
4,Color,Peter Jackson,186,258355354.0,Adventure|Fantasy,The Hobbit: The Desolation of Smaug,2013,English,USA,225000000.0,7.9,"Aidan Turner,Adam Brown,James Nesbitt",83000


# Memeriksa Data

In [ ]:
df.shape

(99, 13)

Memeriksa banyak data (baris) dan atribut (kolom) yang tersedia sebelum melanjutkan ke langkah preprocessing atau analisis. dataset "Movie" memiliki 99 baris dan 13 kolom.

Memeriksa tipe data setiap kolom dalam DataFrame pada dataset "Movie".

In [ ]:
df.dtypes

,0
color,object
director_name,object
duration,int64
gross,float64
genres,object
movie_title,object
title_year,int64
language,object
country,object
budget,float64


Memanfaatkan fungsi .isnull().sum(), Kita dapat mengetahui jumlah missing data pada masing-masing kolom di dataset “Movie”.

In [ ]:
import numpy as np

# cek banyaknya missing value pada masing-masing kolom
print(df.isnull().sum())

color                   11
director_name           11
duration                 0
gross                    8
genres                   1
movie_title              0
title_year               0
language                 0
country                  0
budget                   4
imdb_score               0
actors                   0
movie_facebook_likes     0
dtype: int64


# Membersihkan Data

dropna() digunakan untuk menghapus baris (rows) yang memiliki nilai kosong (NaN).
Parameter subset=['gross', 'budget'] berarti hanya akan memeriksa kolom gross dan budget.
Jadi, jika salah satu atau kedua kolom tersebut berisi NaN pada suatu baris, baris tersebut akan dihapus dari DataFrame.

In [ ]:
# Hapus baris dengan NaN di kolom penting: 'gross' dan 'budget'
df = df.dropna(subset=['gross', 'budget'])

Mengubah semua teks di kolom color menjadi huruf kecil agar konsisten dengan memanfaatkan fungsi .str.lower().

In [ ]:
# Atasi nilai yang tidak konsisten
# samakan 'Color' vs 'color'
if 'color' in df.columns:
    df['color'] = df['color'].str.lower().str.strip()

Pada dataset “Movie” missing data memiliki nilai 'N/A' atau string kosong " ". Kita dapat melakukan replace 'N/A' atau string kosong " " dengan NaN (Not a number) yang merupakan penanda missing value dalam Python. Kita dapat mengganti data menggunakan fungsi berikut:

.replace(x, y, inplace=True) — dimana x-value akan diganti dengan y-value.



In [ ]:
# Ganti nilai tidak standar seperti 'N/A' atau string kosong menjadi NaN
df.replace(['N/A', 'n/a', '', 'na'], np.nan, inplace=True)

membersihkan data numerik agar tidak ada nilai negatif pada kolo budget, gross, duration, atau imdb_score dengan memanfaatkan fungsi .astype(float) >= 0 (menyaring baris agar hanya menyisakan nilai >= 0 (membuang baris dengan nilai negatif))

In [ ]:
# Pastikan tidak ada angka negatif di kolom numerik tertentu
num_cols = ['budget', 'gross', 'duration', 'imdb_score']
for col in num_cols:
    if col in df.columns:
        df = df[df[col].astype(float) >= 0]

# Transformasi Data

Mengubah tipe data menjadi format numerik (float atau int) untuk kolom gross dan budget menggunakan fungsi .astype().

In [ ]:
# Ubah tipe data kolom budget dan gross menjadi numerik
df['budget'] = df["budget"].astype("float")
df['gross'] = df["gross"].astype("float")

Mengubah kolom genres yang berisi beberapa kategori dalam satu sel menjadi beberapa kolom biner (0/1) dengan metode one-hot encoding. Proses ini memastikan nilai kosong diganti dengan string kosong, kemudian memecah genre yang dipisahkan tanda | dengan fungsi .str.get_dummies(sep='|', dan akhirnya menambahkan kolom hasil pecahan ke DataFrame.

In [ ]:
# Pisahkan genres menjadi beberapa kolom (one-hot encoding)
if 'genres' in df.columns:
    # pastikan tidak ada NaN
    df['genres'] = df['genres'].fillna('')
    # buat dummy variables dari genre yang dipisahkan '|'
    genre_dummies = df['genres'].str.get_dummies(sep='|')
    # gabungkan kembali ke dataframe
    df = pd.concat([df, genre_dummies], axis=1)

Menormalisasi teks pada semua kolom bertipe string (object). Setiap nilai diubah menjadi string, dikonversi ke huruf kecil, dibersihkan dari spasi di awal/akhir, dan spasi ganda di tengah teks diganti menjadi satu spasi.

In [ ]:
# Normalisasi teks untuk semua kolom bertipe object (string)
for col in df.select_dtypes(include='object').columns:
    df[col] = (
        df[col]
        .astype(str)                # pastikan tipe string
        .str.lower()                # ubah jadi huruf kecil
        .str.strip()                # hapus spasi awal/akhir
        .str.replace(r'\s+', ' ', regex=True)  # ganti spasi ganda jadi satu
    )

# Penyimpanan Data

Terakhir, menyimpan dataset hasil preprocessing ke file CSV baru bernama movie_dataset_cleaned.csv

In [ ]:
output_path = "movie_dataset_cleaned.csv"
df.to_csv(output_path, index=False)

print(f"\nData setelah preprocessing disimpan ke {output_path}")


Data setelah preprocessing disimpan ke movie_dataset_cleaned.csv
